In [13]:
# Import necessary libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
from animal_shelter import AnimalShelter  # Ensure this matches your CRUD module

# Database Connection
username = "aacuser"
password = "yourPasswordHere"
db = AnimalShelter(username, password)

def get_data(filter_query={}):
    df = pd.DataFrame.from_records(db.read(filter_query))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df

# Initialize app
app = JupyterDash(__name__)

# Encode logo image


image_filename = 'Grazioso Salvare Logo.png'  # Use correct filename
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()


# Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Animal Shelter Dashboard'))),
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'}),
    html.Hr(),

    # Filtering Options
    html.Div([
        dcc.Dropdown(id='filter-type', options=[
            {'label': 'All Animals', 'value': 'all'},
            {'label': 'Dogs', 'value': 'Dog'},
            {'label': 'Cats', 'value': 'Cat'}
        ], value='all', placeholder='Filter by Species'),
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in get_data().columns],
        data=get_data().to_dict('records'),
        row_selectable='single',
        style_table={'overflowX': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    
    # Graph & Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callbacks
@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {} if filter_type == 'all' else {"animal_type": filter_type}
    return get_data(query).to_dict('records')

@app.callback(Output('graph-id', 'children'), [Input('datatable-id', 'derived_virtual_data')])
def update_graphs(viewData):
    df = pd.DataFrame(viewData)
    if df.empty:
        return html.P("No data available.")
    fig = px.pie(df, names='breed', title='Animal Breed Distribution')
    return dcc.Graph(figure=fig)

@app.callback(Output('map-id', 'children'),
              [Input('datatable-id', 'derived_virtual_data'),
               Input('datatable-id', 'derived_virtual_selected_rows')])
def update_map(viewData, index):
    if not viewData or index is None:
        return html.P("No location data available.")
    df = pd.DataFrame(viewData)
    row = index[0] if index else 0
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[df.iloc[row]['location_lat'], df.iloc[row]['location_long']],
                  children=[dl.Tooltip(df.iloc[row]['breed']),
                            dl.Popup([html.H1("Animal Name"), html.P(df.iloc[row]['name'])])])
    ])



app.run_server(debug=True)


Dash app running on http://127.0.0.1:10087/


# README

# Project Title: Animal Shelter Dashboard

Overview
This project is a web-based dashboard for an animal shelter that allows users to view, filter, and visualize data about the animals in the shelter. The dashboard is built using Dash, Plotly, and Dash Leaflet to display interactive data and maps. The dashboard provides an interface to filter by animal species and view related charts and maps.



# Table of Contents
1. Project Setup
2. Features
3. Installation Instructions
4. Usage
5. Screenshots
6. Troubleshooting

# Project Setup

Prerequisites
Befoe beginning , ensure you have the following installed:

1. Python 3.7+
2. pip (Python package manager)
3. MongoDB (connect to  MongoDB database)
4. Dash and related libraries (Dash Leaflet, Plotly, Pandas)

# Install Dependencies

To set up the project, install the required Python libraries.

install these dependencies using pip:
pip install dash dash-leaflet plotly pandas pymongo


# MongoDB Setup

1.Ensure your MongoDB is running and accessible, either locally or remotely.
2. Replace the credentials in the script with your MongoDB username and password.
3. Ensure that your database contains the necessary animal shelter data.


# Features
1. Data Table: Displays a table of animals in the shelter with filtering options (species-based).
2. Breed Distribution Pie Chart: Visualizes the breed distribution of the animals using a pie chart.
3. Map: Displays the animal locations on an interactive map using Dash Leaflet.
4. Filters: Allows the user to filter the animals by species (e.g., Dogs, Cats).


# Installation Instructions
Download the project files.
Installed the required libraries listed above.
Set up your MongoDB database with the appropriate data (animal details, including latitude and longitude).
Replaced the username, password, and other database connection details in the code with your actual credentials.


# Usage

Open your Jupyter Notebook environment.

In the Jupyter Notebook, run the script cell by cell or click Run All to execute the code.

Once the code is executed, the output will display in the form of a Dash app running on your local machine.

You should see a message like this in the notebook output:

Dash app running on http://127.0.0.1:10087/
Open a web browser and navigate to http://127.0.0.1:10087/ to view the dashboard.

On the dashboard:

- Use the filtering dropdown to filter animals by species (Dog, Cat, or All).
- View the breed distribution pie chart.
- See the map showing the locations of animals by clicking on the rows in the data table.


# Troubleshooting
If encounter issues, here are some common solutions:

Authentication Errors: Ensure your MongoDB credentials are correct and that your MongoDB server is accessible.
Missing Columns: If certain data fields (e.g., latitude/longitude) are missing, check your MongoDB database to ensure all records are complete.